In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

yr2020_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2020.csv')
yr2020_df['fips'] = yr2020_df['fips'].fillna(0).astype(np.int64)
yr2020_df['date'] = pd.to_datetime(yr2020_df['date'], yearfirst=True)
yr2021_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2021.csv')
yr2021_df['fips'] = yr2021_df['fips'].fillna(0).astype(np.int64)
yr2021_df['date'] = pd.to_datetime(yr2021_df['date'], yearfirst=True)
yr2022_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2022.csv')
yr2022_df['fips'] = yr2022_df['fips'].fillna(0).astype(np.int64)
yr2022_df['date'] = pd.to_datetime(yr2022_df['date'], yearfirst=True)
us_covid_df = pd.concat([yr2020_df, yr2021_df, yr2022_df])
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
2454529,2022-05-30,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2454530,2022-05-31,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2454531,2022-06-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2454532,2022-06-02,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
62887,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
62889,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
62891,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
62893,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
62895,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1484540,2022-05-30,Arkansas,Arkansas,5001,5713,78.00,5000,Arkansas,AR,Arkansas,State,3017804
1484542,2022-05-31,Arkansas,Arkansas,5001,5713,78.00,5000,Arkansas,AR,Arkansas,State,3017804
1484544,2022-06-01,Arkansas,Arkansas,5001,5715,78.00,5000,Arkansas,AR,Arkansas,State,3017804
1484546,2022-06-02,Arkansas,Arkansas,5001,5719,78.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2454529,2022-05-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2454530,2022-05-31,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2454531,2022-06-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2454532,2022-06-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2022-06-03') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
864,2022-06-03,Snohomish,Washington,166412,1161.00,53061,WA,County,822083,20242.72,141.23
1726,2022-06-03,Cook,Illinois,1242012,14991.00,17031,IL,County,5150233,24115.65,291.07
2587,2022-06-03,Orange,California,619971,7045.00,6059,CA,County,3175692,19522.39,221.84
3447,2022-06-03,Maricopa,Arizona,1305383,17384.00,4013,AZ,County,4485414,29102.84,387.57
4307,2022-06-03,Los Angeles,California,2991355,32154.00,6037,CA,County,10039107,29797.02,320.29
...,...,...,...,...,...,...,...,...,...,...,...
2452261,2022-06-03,Wheeler,Oregon,247,3.00,41069,OR,County,1332,18543.54,225.23
2452860,2022-06-03,King,Texas,52,0.00,48269,TX,County,272,19117.65,0.00
2453428,2022-06-03,Esmeralda,Nevada,108,3.00,32009,NV,County,873,12371.13,343.64
2453992,2022-06-03,Loving,Texas,212,1.00,48301,TX,County,169,125443.79,591.72


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
864,2022-06-03,Snohomish,Washington,166412,1161.00,53061,WA,County,822083,20242.72,141.23,141.23,20242.72
1726,2022-06-03,Cook,Illinois,1242012,14991.00,17031,IL,County,5150233,24115.65,291.07,291.07,24115.65
2587,2022-06-03,Orange,California,619971,7045.00,6059,CA,County,3175692,19522.39,221.84,221.84,19522.39
3447,2022-06-03,Maricopa,Arizona,1305383,17384.00,4013,AZ,County,4485414,29102.84,387.57,387.57,29102.84
4307,2022-06-03,Los Angeles,California,2991355,32154.00,6037,CA,County,10039107,29797.02,320.29,320.29,29797.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452261,2022-06-03,Wheeler,Oregon,247,3.00,41069,OR,County,1332,18543.54,225.23,225.23,18543.54
2452860,2022-06-03,King,Texas,52,0.00,48269,TX,County,272,19117.65,0.00,0.00,19117.65
2453428,2022-06-03,Esmeralda,Nevada,108,3.00,32009,NV,County,873,12371.13,343.64,343.64,12371.13
2453992,2022-06-03,Loving,Texas,212,1.00,48301,TX,County,169,125443.79,591.72,591.72,125443.79


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1553087,2022-06-03,Hale,Alabama,4812,106.00,1065,AL,County,14651,32844.17,723.50,723.50,32844.17,1
1309676,2022-06-03,Winston,Alabama,7609,132.00,1133,AL,County,23629,32201.96,558.64,558.64,32201.96,2
899406,2022-06-03,Franklin,Alabama,9943,139.00,1059,AL,County,31362,31703.97,443.21,443.21,31703.97,3
1070310,2022-06-03,Clay,Alabama,4115,82.00,1027,AL,County,13235,31091.80,619.57,619.57,31091.80,4
1552291,2022-06-03,Fayette,Alabama,5003,96.00,1057,AL,County,16302,30689.49,588.88,588.88,30689.49,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098188,2022-06-03,Big Horn,Wyoming,2605,61.00,56003,WY,County,11790,22095.00,517.39,517.39,22095.00,19
1305681,2022-06-03,Johnson,Wyoming,1859,19.00,56019,WY,County,8445,22013.03,224.99,224.99,22013.03,20
1846944,2022-06-03,Lincoln,Wyoming,4001,31.00,56023,WY,County,19830,20176.50,156.33,156.33,20176.50,21
1482157,2022-06-03,Sublette,Wyoming,1936,27.00,56035,WY,County,9831,19692.81,274.64,274.64,19692.81,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
1212081,2022-06-03,Lowndes,Alabama,2645,77.00,1085,AL,County,9726,27195.15,791.69,791.69,27195.15,32,1
1210481,2022-06-03,Crenshaw,Alabama,3860,100.00,1041,AL,County,13772,28027.88,726.11,726.11,28027.88,24,2
1553087,2022-06-03,Hale,Alabama,4812,106.00,1065,AL,County,14651,32844.17,723.50,723.50,32844.17,1,3
511921,2022-06-03,Walker,Alabama,18861,447.00,1127,AL,County,63521,29692.54,703.70,703.70,29692.54,9,4
1067907,2022-06-03,Butler,Alabama,5094,129.00,1013,AL,County,19448,26192.92,663.31,663.31,26192.92,39,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305681,2022-06-03,Johnson,Wyoming,1859,19.00,56019,WY,County,8445,22013.03,224.99,224.99,22013.03,20,19
1800216,2022-06-03,Uinta,Wyoming,5681,39.00,56041,WY,County,20226,28087.61,192.82,192.82,28087.61,7,20
1846944,2022-06-03,Lincoln,Wyoming,4001,31.00,56023,WY,County,19830,20176.50,156.33,156.33,20176.50,21,21
1207281,2022-06-03,Albany,Wyoming,10081,49.00,56001,WY,County,38880,25928.50,126.03,126.03,25928.50,14,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,141.23,20242.72,32,17
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,141.23,20242.72,32,17
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,141.23,20242.72,32,17
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,141.23,20242.72,32,17
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,141.23,20242.72,32,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2448880,2022-05-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2448881,2022-05-31,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2448882,2022-06-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2448883,2022-06-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1546643,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,32844.17,3,1,1.00,nan
1546644,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,32844.17,3,1,0.00,nan
1546645,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,32844.17,3,1,0.00,nan
1546646,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,32844.17,3,1,0.00,nan
1546647,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,723.50,32844.17,3,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1878468,2022-05-30,Crook,Wyoming,1353,29.00,56011,WY,County,7584,17840.19,382.38,382.38,17879.75,9,23,0.00,0.14
1878469,2022-05-31,Crook,Wyoming,1356,29.00,56011,WY,County,7584,17879.75,382.38,382.38,17879.75,9,23,3.00,0.29
1878470,2022-06-01,Crook,Wyoming,1356,29.00,56011,WY,County,7584,17879.75,382.38,382.38,17879.75,9,23,0.00,0.29
1878471,2022-06-02,Crook,Wyoming,1356,29.00,56011,WY,County,7584,17879.75,382.38,382.38,17879.75,9,23,0.00,0.29


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa['14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa['deaths'] = usa['deaths'].astype(int)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1207229,2020-03-26,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,791.69,27195.15,1,32,1.00,43.86,0.00,nan
1207230,2020-03-27,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,791.69,27195.15,1,32,0.00,43.86,0.00,nan
1207231,2020-03-28,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,791.69,27195.15,1,32,0.00,41.36,0.00,nan
1207232,2020-03-29,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,791.69,27195.15,1,32,0.00,39.71,0.00,nan
1207233,2020-03-30,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,791.69,27195.15,1,32,0.00,36.93,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506244,2022-05-30,Teton,Wyoming,10167,16,56039,WY,County,23464,43330.21,68.19,68.19,43594.44,23,1,0.00,6.64,0.00,0.00
506245,2022-05-31,Teton,Wyoming,10229,16,56039,WY,County,23464,43594.44,68.19,68.19,43594.44,23,1,62.00,8.07,0.00,0.00
506246,2022-06-01,Teton,Wyoming,10229,16,56039,WY,County,23464,43594.44,68.19,68.19,43594.44,23,1,0.00,8.07,0.00,0.00
506247,2022-06-02,Teton,Wyoming,10229,16,56039,WY,County,23464,43594.44,68.19,68.19,43594.44,23,1,0.00,8.07,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2022-06-03') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
611957,2022-06-03,Imperial,California,67855,940,6025,CA,County,181215,37444.47,518.72,518.72,37444.47,1,1,306.00,54.50,3.00,1.29
93167,2022-06-03,Shasta,California,37565,655,6089,CA,County,180080,20860.17,363.73,363.73,20860.17,2,33,84.00,26.93,0.00,0.14
303141,2022-06-03,San Bernardino,California,608303,7664,6071,CA,County,2180085,27902.72,351.55,351.55,27902.72,3,7,1875.00,539.07,23.00,10.07
1482884,2022-06-03,Tuolumne,California,14170,189,6109,CA,County,54478,26010.50,346.93,346.93,26010.50,4,12,120.00,18.36,0.00,0.50
4307,2022-06-03,Los Angeles,California,2991355,32154,6037,CA,County,10039107,29797.02,320.29,320.29,29797.02,5,4,4986.00,4335.79,8.00,5.71
167394,2022-06-03,Tulare,California,138523,1490,6107,CA,County,466195,29713.53,319.61,319.61,29713.53,6,5,547.00,115.07,0.00,0.93
1799319,2022-06-03,Tehama,California,14955,204,6103,CA,County,65084,22978.00,313.44,313.44,22978.00,7,22,36.00,9.14,0.00,0.21
1076670,2022-06-03,Inyo,California,4762,56,6027,CA,County,18039,26398.36,310.44,310.44,26398.36,8,9,26.00,5.36,0.00,0.00
134797,2022-06-03,Stanislaus,California,139246,1694,6099,CA,County,550660,25287.11,307.63,307.63,25287.11,9,15,427.00,142.14,0.00,0.00
974833,2022-06-03,Merced,California,72992,825,6047,CA,County,277680,26286.37,297.10,297.10,26286.37,10,11,219.00,69.64,1.00,0.14


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
611957,2022-06-03,Imperial,California,67855,940,6025,CA,County,181215,37444.47,518.72,518.72,37444.47,1,1,306.00,54.50,3.00,1.29
1308020,2022-06-03,Kings,California,56304,447,6031,CA,County,152940,36814.44,292.27,292.27,36814.44,12,2,138.00,42.93,0.00,0.00
2307346,2022-06-03,Lassen,California,10903,69,6035,CA,County,30573,35662.19,225.69,225.69,35662.19,21,3,25.00,9.00,0.00,0.00
4307,2022-06-03,Los Angeles,California,2991355,32154,6037,CA,County,10039107,29797.02,320.29,320.29,29797.02,5,4,4986.00,4335.79,8.00,5.71
167394,2022-06-03,Tulare,California,138523,1490,6107,CA,County,466195,29713.53,319.61,319.61,29713.53,6,5,547.00,115.07,0.00,0.93
61248,2022-06-03,Madera,California,44565,372,6039,CA,County,157327,28326.35,236.45,236.45,28326.35,19,6,144.00,38.71,0.00,0.14
303141,2022-06-03,San Bernardino,California,608303,7664,6071,CA,County,2180085,27902.72,351.55,351.55,27902.72,3,7,1875.00,539.07,23.00,10.07
381753,2022-06-03,Kern,California,248841,2452,6029,CA,County,900202,27642.80,272.38,272.38,27642.80,14,8,767.00,242.21,3.00,1.21
1076670,2022-06-03,Inyo,California,4762,56,6027,CA,County,18039,26398.36,310.44,310.44,26398.36,8,9,26.00,5.36,0.00,0.00
60429,2022-06-03,Fresno,California,262836,2766,6019,CA,County,999101,26307.25,276.85,276.85,26307.25,13,10,845.00,261.43,1.00,0.43


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
611957,2022-06-03,Imperial,California,67855,940,6025,CA,County,181215,37444.47,518.72,518.72,37444.47,1,1,306.00,54.50,3.00,1.29
93167,2022-06-03,Shasta,California,37565,655,6089,CA,County,180080,20860.17,363.73,363.73,20860.17,2,33,84.00,26.93,0.00,0.14
303141,2022-06-03,San Bernardino,California,608303,7664,6071,CA,County,2180085,27902.72,351.55,351.55,27902.72,3,7,1875.00,539.07,23.00,10.07
1482884,2022-06-03,Tuolumne,California,14170,189,6109,CA,County,54478,26010.50,346.93,346.93,26010.50,4,12,120.00,18.36,0.00,0.50
4307,2022-06-03,Los Angeles,California,2991355,32154,6037,CA,County,10039107,29797.02,320.29,320.29,29797.02,5,4,4986.00,4335.79,8.00,5.71
167394,2022-06-03,Tulare,California,138523,1490,6107,CA,County,466195,29713.53,319.61,319.61,29713.53,6,5,547.00,115.07,0.00,0.93
1799319,2022-06-03,Tehama,California,14955,204,6103,CA,County,65084,22978.00,313.44,313.44,22978.00,7,22,36.00,9.14,0.00,0.21
1076670,2022-06-03,Inyo,California,4762,56,6027,CA,County,18039,26398.36,310.44,310.44,26398.36,8,9,26.00,5.36,0.00,0.00
134797,2022-06-03,Stanislaus,California,139246,1694,6099,CA,County,550660,25287.11,307.63,307.63,25287.11,9,15,427.00,142.14,0.00,0.00
974833,2022-06-03,Merced,California,72992,825,6047,CA,County,277680,26286.37,297.10,297.10,26286.37,10,11,219.00,69.64,1.00,0.14


In [32]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'case_day', '14day_case_avg']]
top10_ca_case_df['case_day'] = top10_ca_case_df['case_day'].astype(int)
top10_ca_case_df['date'] =  pd.to_datetime(top10_ca_case_df['date'], infer_datetime_format=True)
top10_ca_case_df = top10_ca_case_df[(top10_ca_case_df['date'] > "2020-04-30")]
top10_ca_case_df

,date,county,cases,current_crate,case_day,14day_case_avg
42,2020-05-01,Imperial,358,197.56,25,14.50
43,2020-05-02,Imperial,358,197.56,0,14.50
44,2020-05-03,Imperial,358,197.56,0,14.14
45,2020-05-04,Imperial,379,209.14,21,15.64
46,2020-05-05,Imperial,400,220.73,21,14.64
...,...,...,...,...,...,...
8075,2022-05-30,Lassen,10815,35374.35,0,4.57
8076,2022-05-31,Lassen,10815,35374.35,0,3.64
8077,2022-06-01,Lassen,10878,35580.41,63,8.14
8078,2022-06-02,Lassen,10878,35580.41,0,7.93


In [33]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'death_day', '14day_death_avg']]
top10_ca_death_df['death_day'] = top10_ca_death_df['death_day'].astype(int)
top10_ca_death_df['date'] =  pd.to_datetime(top10_ca_death_df['date'], infer_datetime_format=True)
top10_ca_death_df

,date,county,deaths,current_drate,death_day,14day_death_avg
0,2020-03-20,Imperial,0,0.00,0,0.07
1,2020-03-21,Imperial,0,0.00,0,0.07
2,2020-03-22,Imperial,0,0.00,0,0.07
3,2020-03-23,Imperial,0,0.00,0,0.07
4,2020-03-24,Imperial,0,0.00,0,0.07
...,...,...,...,...,...,...
8110,2022-05-30,Merced,824,296.74,0,0.07
8111,2022-05-31,Merced,824,296.74,0,0.07
8112,2022-06-01,Merced,824,296.74,0,0.43
8113,2022-06-02,Merced,824,296.74,0,0.43


In [34]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [35]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)